In [1]:
% load_ext autoreload
% autoreload 2

In [4]:
import torch
import utils
import vanilla_train
import vanilla_seq2seq
from nltk.translate import bleu_score
import cvae

In [3]:
train_iter, val_iter, test, DE, EN = utils.torchtext_extract()

In [5]:
model_name = "load"

model_path = "vae_lstmprojattn_wt_100/21.pt"

gpu = True

num_layers = 2
embed_size = 300
hidden_size = 300

num_epochs=10

In [ ]:
model = torch.load(model_path)

print("Number of parameters: {}".format(utils.count_parameters(model)))

In [ ]:
new_model = cvae.CVAE(len(DE.vocab), len(EN.vocab), embed_size, hidden_size, latent_size, num_layers, dpt)
if gpu:
    new_model.cuda()

In [ ]:
y = utils.generate(model, val_iter, EN, 10, 100, gpu)

for s in y:
    utils.strip(s)

In [ ]:
index = 0
avg_bleu = 0

for batch in val_iter:
    trg = batch.trg
    for i in range(trg.size(1)):
            t = []
            for word in trg[:, i]:
                t += [EN.vocab.itos[word]]
            utils.strip(t)
            bleu = bleu_score.sentence_bleu([t], y[index], emulate_multibleu=True)
            
            print("TRG: " + str(t))
            print("OUT: " + str(y[index]))
            print("BLEU: " + str(bleu))
            
            avg_bleu += bleu
            index += 1

print("AVG BLEU: " + str(avg_bleu / index))